<a href="https://colab.research.google.com/github/CatherineBraguina/-/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%8D%D1%84%D1%84%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%BF%D1%80%D0%BE%D0%BC%D0%BE_%D0%B0%D0%BA%D1%86%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Анализ эффективности промо-акций

*Методы анализа:*
-  ⁠*Сравнение продаж до и после акции.*
-  ⁠*Возврат на инвестиции (ROI):* ((Выручка от промо - Затраты на промо) / Затраты на промо) * 100
-  ⁠*Анализ прироста продаж:* Сравнение объемов продаж до, во время и после акции.

In [1]:
#импорт библиотек
import pandas as pd

In [2]:
# создаем данные и фрейм
data = {
    'date': pd.date_range(start='2023-01-01', periods=30),
    'sales': [100, 110, 105, 200, 220, 210, 300, 320, 310, 150] + [100] * 20
}
df = pd.DataFrame(data)

In [3]:
# Расчет средней продажи до, во время и после акции
before_promo = df['sales'][:3].mean()
during_promo = df['sales'][3:7].mean()
after_promo = df['sales'][7:].mean()

print(f"Средние продажи до акции: {before_promo}")
print(f"Средние продажи во время акции: {during_promo}")
print(f"Средние продажи после акции: {after_promo}")


Средние продажи до акции: 105.0
Средние продажи во время акции: 232.5
Средние продажи после акции: 120.8695652173913


In [4]:
# ROI
cost_of_promo = 1000 # Затраты на акцию
revenue_from_promo = (during_promo * 4) - cost_of_promo
roi = (revenue_from_promo / cost_of_promo) * 100
print(f"ROI: {roi}%")

ROI: -7.000000000000001%


# Зачем считать ROI ?

1. Оценить эффективность затрат на рекламу.
2. Спланировать маркетинговый бюджет.
3. Понять, в каких каналах трафика бизнесу выгоднее продавать товары и услуги.

# ! РАЗНИЦА ROI, ROMI, ROAS !

- ROAS (возврат на рекламные расходы)
- ROMI (возврат на инвестиции в маркетинг)
- ROI (возврат на инвестиции)

ROI = ROMI (затраты на маркетинг: на рекламу, печать листовок, написание текстов райтерами и так далее. А вот, например, затраты на зарплату сотрудников или производство продукта не учитываются) + ROAS (расходы на рекламу и доходы, которые были с неё получены)

### 3. Прогнозирование следующих промо

Для прогнозирования может быть использован метод временных рядов или регрессионный анализ. Один из распространенных методов — линейная регрессия.

In [5]:
#импорт библиотек
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [6]:
#создаем данные
data = {
    'promo_days': [1, 2, 3, 4, 5, 6, 7],
    'sales': [110, 150, 130, 220, 300, 270, 320]
}
df = pd.DataFrame(data)

In [7]:
# Разделение на признаки и целевую переменную
X = df[['promo_days']]
y = df['sales']

In [8]:
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Создание модели
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [10]:
# Прогнозирование
future_promos = np.array([[8], [9], [10]]).reshape(-1, 1)
predictions = model.predict(future_promos)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [11]:
display(f"Прогноз продаж на следующие акции: {predictions}")

'Прогноз продаж на следующие акции: [377. 420. 463.]'

### 2. Расчет эффекта каннибализации

Эффект каннибализации возникает, когда акционные товары "отнимают" продажи у других товаров.

*Пример:*
•⁠  ⁠Пусть у нас есть два продукта: A и B. Мы запускаем акцию на продукт A и анализируем, как это влияет на продажи продукта B.



In [ ]:
WITH sales_data AS (
    SELECT product,
           date,
           SUM(sales) AS total_sales
    FROM sales
    WHERE date BETWEEN '2023-01-01' AND '2023-01-30'
    GROUP BY product, date
),
promo_effect AS (
    SELECT
        date,
        SUM(CASE WHEN product = 'A' THEN total_sales ELSE 0 END) AS sales_A,
        SUM(CASE WHEN product = 'B' THEN total_sales ELSE 0 END) AS sales_B
    FROM sales_data
    GROUP BY date
)
SELECT
    AVG(sales_A) AS avg_sales_A,
    AVG(sales_B) AS avg_sales_B,
    (AVG(sales_B) - AVG(sales_B) OVER (PARTITION BY date ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)) AS cannibalization_effect
FROM promo_effect;
 ⁠


 # Бонус - расчет доппоказателей
 - CPO (стоимость привлечения одной покупки)
 - средний чек,
 - LTV (пожизненная ценность клиента),
 - доход,
 - расход
 - выручка с рекламы

•⁠  ⁠*CPO* рассчитывается как общая стоимость рекламы, деленная на количество продаж.
•⁠  ⁠*Средний чек* — это общий доход, деленный на количество продаж.
•⁠  ⁠*LTV* (пожизненная ценность клиента) здесь предполагается как средний чек, умноженный на 12 (для случаев, когда клиент совершает покупки 12 раз в год).
•⁠  ⁠*ROAS, **ROMI* и *ROI* рассчитываются на основе общей выручки и расходов на рекламу и показывают рентабельность рекламных инвестиций.

### Разница между показателями
•⁠  ⁠*CPO* показывает, сколько стоит привлечение одной продажи, в то время как *ROMI, **ROAS* и *ROI* показывают окупаемость бюджета, потраченного на рекламу.
•⁠  ⁠*Средний чек* и **

Предположим, у нас есть следующие таблицы:
-  ⁠⁠ sales ⁠ — продажи (id, customer_id, amount, date)
-  ⁠⁠ ads ⁠ — реклама (id, cost, revenue, date)

In [ ]:
SELECT
    SUM(s.amount) AS total_revenue,
    COUNT(s.id) AS total_sales,
    SUM(a.cost) AS total_ad_cost,
    SUM(a.revenue) AS total_ad_revenue
FROM
    sales s
JOIN
    ads a ON DATE(s.date) = DATE(a.date)
WHERE
    s.date BETWEEN '2023-01-01' AND '2023-12-31';
 ⁠

с помощью Python считаем показатели

In [ ]:
import sqlite3

# Создаем подключение к базе данных (предположим, что у нас SQLite).
connection = sqlite3.connect('your_database.db')
cursor = connection.cursor()

# Выполняем SQL запрос для извлечения необходимых данных.
query = """
SELECT
    SUM(s.amount) AS total_revenue,
    COUNT(s.id) AS total_sales,
    SUM(a.cost) AS total_ad_cost,
    SUM(a.revenue) AS total_ad_revenue
FROM
    sales s
JOIN
    ads a ON DATE(s.date) = DATE(a.date)
WHERE
    s.date BETWEEN '2023-01-01' AND '2023-12-31';
"""
cursor.execute(query)

# Извлекаем результаты.
result = cursor.fetchone()
total_revenue, total_sales, total_ad_cost, total_ad_revenue = result

# Расчеты
def calculate_metrics(total_revenue, total_sales, total_ad_cost):
    # CPO (стоимость привлечения одной покупки)
    cpo = total_ad_cost / total_sales if total_sales > 0 else 0

    # Средний чек
    average_check = total_revenue / total_sales if total_sales > 0 else 0

    # LTV (пожизненная ценность клиента)
    ltv = average_check * 12  # Предположим, что средний клиент совершает покупку 12 раз в год

    # ROAS (возврат на рекламные расходы)
    roas = total_ad_revenue / total_ad_cost if total_ad_cost > 0 else 0

    # ROMI (возврат на инвестиции в маркетинг)
    romi = (total_revenue - total_ad_cost) / total_ad_cost if total_ad_cost > 0 else 0

    # ROI (возврат на инвестиции)
    roi = (total_revenue - total_ad_cost) / total_ad_cost if total_ad_cost > 0 else 0

    return {
        "CPO": cpo,
        "Average Check": average_check,
        "LTV": ltv,
        "ROAS": roas,
        "ROMI": romi,
        "ROI": roi
    }

# Вычисляем показатели
metrics = calculate_metrics(total_revenue, total_sales, total_ad_cost)

# Выводим результаты
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

# Закрываем соединение
connection.close()
 ⁠